# Reading the data and adding Enrichments

#### In this section we'll read the csv files extracted from the production environment
#### through Athena & Snowflake.

#### The data will have it's first feature selection before enrichments and one-hot - coding
#### Eventually, the processed data will be forwarded to initial exploration.

### modules imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time as time
import tqdm 
import os
from scipy.stats import chisquare

In [2]:
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler

In [3]:
import seaborn as sns
import math as math
import warnings

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from IPython.display import Image
from sklearn import tree
from sklearn import svm
from sklearn.tree import export_graphviz
from subprocess import check_call
pd.set_option('display.max_columns', 500)

In [5]:
%run funcs.py

ERROR:root:File `'funcs.py'` not found.


### Setting Path definitions

In [6]:

Input_Path = r'C:\Users\Persona.ly_DS1\Desktop\Technion\TABLES'
Output_Path = r'C:\Users\Persona.ly_DS1\Desktop\Technion\Output'

#### global varaibales definition - Hour Groups and weekdays

In [7]:
global HGroups, Weekdays

Weekdays = {0:'1',1:'2',2:'3',3:'4',4:'5',5:'6',6:'7'}



## Useful functions for the process

In [8]:
def RatingGroup(a):
    if a==np.nan:
        return "Other"
    elif 0<a<4:
        return "0-4"
    elif 4<a<4.5:
        return "4-4.5"
    else:
        return "4.5-5"

In [9]:
def device_screen_size(a):
    if 0<a<4.5:
        return 'Lowcost'
    elif 4.5<a<5.5:
        return 'Standard'
    elif 5.5<a<7.0:
        return 'Phablet'
    elif a>7:
        return 'Tablet'
    pass
        

In [10]:
def hourgroup(a):
#"""divides the day to 3 shifts of 8 hours"""
    if np.isnan(a) :
        return ''
    HGroups = {'16-23':list(range(16,24)),'0-7':list(range(0,8)),'8-15':list(range(8,16))}
    for key, values in HGroups.items():
        if int(a) in values:
            return key

###   Often different add providers - supply different names for the same OS Version. This function repairs it: example osv_normalizer(6, 6.0, 6.0.0) = 6
           

In [11]:
import string
def osv_normalizer(x):
    try:
        if math.isnan(float(x)) :
            return 'Other'
    except:
       warnings.filterwarnings("ignore")  
    norm = x
    Flag = True
    while Flag:
        if (norm[-1] == '0') and (not norm[-2].isnumeric()):
            norm=norm[:-2] 
        else:
            Flag = False           
    return norm    

## Read Files ( DataSet, Apps and category enrichment, Region enrichement)

In [12]:
os.chdir(Input_Path)
AppCategory = pd.read_csv('AppCategory.csv')
us_tz = pd.read_csv('us_tz.csv')
AppRating = pd.read_csv('AppRating.csv')

In [13]:
df = pd.read_csv('Monopoly_388_01_2019_imp_click_inst.csv',low_memory = False)


#### Validation Checks

Filter only Billable values such as impression/ install 
(not all event types are relevant for the analysis)

This event is stored in column - Type.
Some impressions event has the character - 
yg_impr == 1 meaning the impression should be ignored
Some install events also has a character
ig_install == 1 meaning the installations should be ignored

In [14]:
df.type.value_counts()

impression      662616
v_start         517259
v_companion     464870
v_end           450312
click            44879
v_comp_click     37026
v_error           5417
install           2892
app_event            3
Name: type, dtype: int64

In [15]:
#Count no. of impressions / installed to be ignored (invalid data)
df.ig_impr.sum() , df.ig_install.sum()

(118403.0, 1.0)

In [16]:
df= df[(df.install == 1 )| (df.impr == 1)]
df.type.value_counts()

impression    544213
install         2891
Name: type, dtype: int64

In [17]:
df[['install','impr','installprice']]  = df[['install','impr','installprice']].fillna(value=0)

Filt = df.bidid.isin(set(df.bidid[df.install == 1]))
Filt2 = df.type == 'impression'

imprs_for_installs = df[['bidid','imprprice']][Filt].groupby('bidid').sum()
df.imprprice[Filt & ~Filt2] = df[Filt & ~Filt2].join(imprs_for_installs,on ='bidid',rsuffix = '_').imprprice_
df = df[~(Filt & Filt2)]

C:\Users\Persona.ly_DS1\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [18]:
df.imprprice = df.imprprice.fillna(0)

TOTAL INCOME FROM THE CAMPAIGN - $ 14455
TOTAL COST OF THE CAMPAIGN - $10696.87

In [19]:
df['Impression_Price'] = df.imprprice/1000 # calculate the real cost of the impression
df.drop(columns=['imprprice'],axis = 1,inplace=True)

In [20]:
colist = df.columns.tolist()
colist = sorted(colist)

In [21]:
not_to_use = [
                'g_end',
                'click',
                'devicetype'
                'g_start',
                'g_view',
                'gender',
                'geooffset',
                'v_companion',
                'v_companion_click',
                'v_companion_install',
                'v_end',
                'v_error',
                'v_render',
                'v_start',
                'v_start_time',
                 'width',
                'win',
                'winprice',
                'year',
                'ig_click',
                'ig_g_end',
                'ig_g_start',
                'ig_g_view',
                'ig_impr',
                'ig_install',
                'ig_v_companion',
                'ig_v_companion_click',
                'ig_v_end',
                'ig_v_error',
                'ig_v_render',
                'ig_v_start',
                'ig_win',
                'ignorereason',
                'os',
                'DeviceType',
                'Device Type',
                'appevent',
                'appeventcurrency',
                'appeventrevenue',
                'appeventtime',
                'appeventtoken',
                'avgwinpricerange',
                'g_start',
                'clickprice',
                'contype',
                'bidder','bidderid','country','cid','fraudlogixscore','ua','ip',
                'hour','day','bidfloor','devicetype','iosbuildid','siteid','tz',
                'utc_diff', 'Region','state_name','agebucket','fctype','dailyfc','lifetimefc',
                'displaymanager','netspeed','bidfloor'
                
            ]
df = df[[x for x in colist if x not in not_to_use]]

## Data Enrichments from Tables.

In [22]:
df = df.join(AppCategory.set_index('plBundle'), on='bundle') 
df = df.join(us_tz.set_index('abbr'), on='state',rsuffix=' ')
df = df.join(AppRating[['plBundle','bundleAvgUserRating']].set_index('plBundle'), on='bundle')

## Time Enrichement:
creating local hour and day variables for the data out of utc. 
using us_timezone table with time difference from utc according to state

In [24]:
time.strftime('%m/%d/%Y %H:%M:%S')
df['time'] = df['time']/1000.# careful not to run more than once on itteration - converting ml-seconds to seconds
df['date'] = pd.to_datetime(df['time'],unit='s') # convert utc to regular datetime display

In [25]:
df['utc_diff'] = abs(df['utc_diff']) # in order to create time delta of hours
df['time_diff'] = pd.to_timedelta(df['utc_diff'],'h')#convert no. of hours (int) to timedelta of x hours
df['local time'] = df['date'] - df['time_diff'] #subtract the hours to get local time

In [26]:
df['Local_hour'] = df['local time'].apply(lambda x: x.hour) # create a local hour variable
df['Local_day_of_week'] = df['local time'].apply(lambda x: x.weekday())# create a local day variable (may differ between state to state at a certain timestamp)

In [27]:
#normalizing the amount of known apps for the user
app_min = df.apps.min()
app_max = df.apps.max()
df['normal_apps'] = df.apps.apply(lambda x: (x-app_min)/(app_max - app_min))



In [28]:
#normalizing the amount of known ssps the user has visited.
ssp_min = df.ssps.min()
ssp_max = df.ssps.max()
df['normal_ssps'] = df.ssps.apply(lambda x: (x-ssp_min)/(ssp_max - ssp_min))


adaptation of the columns name to the csv coefficiency model

In [29]:
df.osv = df.osv.apply(osv_normalizer)

In [30]:
rate_min = df.bundleAvgUserRating.min()
rate_max = df.bundleAvgUserRating.max()
df['normal_rate'] = df.bundleAvgUserRating.apply(lambda x: (x-rate_min
                                                           )/(rate_max - rate_min))

In [ ]:
df['isLandscape'] = 1 * (df.devw > df.devh) # Creating an orientation variable while using the phone
df = df.drop(columns=['devh','devw'])

In [ ]:
df['log2ofstorage'] = df.uamaxinternalstorage.apply(math.log2)# converting internal storage to log 2 for easy manipulation
df = df.drop(columns='uamaxinternalstorage')

In [ ]:
df.columns